# PANDA: Predictive Automation of Novel Defect Anomalies

Copyright 2024, Battelle Energy Alliance, LLC, ALL RIGHTS RESERVED

This program (Program) utilizes YOLOv8 under GNU Affero General Public License v3.0 (AGPL-3.0). For more information about YOLOv8, see <https://docs.ultralytics.com/models/yolov8/>. For more information about the AGPL-3.0 license, see https://gnu.org/licenses/. You should have received a copy of the GNU AGPL-3.0 license along with this Program. If not, you may find a copy of it at <https://www.gnu.org/licenses/agpl-3.0.en.html#license-text>.

 

The YOLOv8 is free software. You can redistribute it and/or modify it under the terms of GNU AGPL-3.0 as published by the Free Software Foundation, either version 3 or the License or (at your option) any later version.

 

The portion of the Program that is not YOLOv8 is owned by Battelle Energy Alliance (BEA), (Copyright 2024 BEA) and the source code or instruction sets for running this portion of the program, along with the source code for YOLOv8, are made available to the user upon running the Program. This Program (including the YOLOv8 portion and the BEA portion) is licensed to the user under AGPL-3.0 and can be used according to that license for so long as the user is in compliance with that license.

### Libraries and version numbers tested: 
python, version number: 3.11.5  
opencv-python, version number: 4.9.0.80  
numpy, version number: 1.26.3  
shapely, version number: 2.0.2  
matplotlib, version number: 3.8.2   
ultralytics, version number: 8.0.234  
scikit-learn, version number: 1.3.2  
 PyYAML, version number: 6.0.1  
tensorflow, version number: 2.15.0.post1  
pillow, version number: 10.2.0


### Dataset Preparation and Folder Structure

Begin by downloading `nature_img_used.zip` into the same directory as this code, and unzipping it. In it you will find three folders and a YAML file.


`original_images` contains the raw images obtained from other studies before pre-processing is applied. 
`artery_img_used` includes images and labels from `"Dataset for Automatic Region-based Coronary Artery Disease Diagnostics Using X-Ray Angiography Images"`. Note that this dataset includes multiple classes of arteries that  will be changed to all be 0 aka `loop`. Only 39 images were randomly selected from this dataset to match the number from the loop-heavy `Shen et al.`.

`shen_img_used_loops` includes two images from `"Multi defect detection and analysis of electron microscopy images with deep learning"` and will be referred to elsewhere in this code as `Shen et al.`. The two images are further separated into their own sub-directories due to the differing augmentations applied to them.

`ma956_ods_alloy_dataset` includes images that were obtained as part of NSUF-CINR-18-14787. These images are of a commercially used MA956 ODS alloy that was irradiated. Post irradiation characterization was carried out at Microscopy and Characterization Suite (MaCS), Center of Advanced Energy Studies (CAES). 

The labels of both `shen_img_used_loops` and `ma956_ods_alloy_dataset` were manually annotated during this study using Label Studio, and exported to YOLO format. The original ground-truth labels of `Shen et al.` used a format incompatible with training on YOLO, and thus were re-labelled by this study's researchers. Due to the limited number of annotated images, augmentation techniques are used to create more training images from this select number.


The test images in `test_img` are not trained on, and are predicted on by the trained models for analysis. 

The same goes for `shen_test` which are images from Shen et al. that are not trained on.

Both folders also include the respective ground truth label .txt files. Note that `shen_test` images were not re-labelled, and include the original label format from that study.

Those can be found in`DataSetFinal.zip` which can be downloaded from: `https://figshare.com/articles/dataset/Data_for_Multi_Defect_Detection_and_Analysis_of_Electron_Microscopy_Images_with_Deep_Learning_/8266484`.  


`nature_models.yaml` is a configuration file that defines the directories, classes, and image augmentations used during training.
##### Do note that YAML files only accept direct PATHs, and thus you must fill in your working directory (aka the directory where you unzipped nature_img_used.zip) at the beginning of the `path` parameters in the file.
##### Only one of the two `path` parameters should remain uncommented for a given training, and they should be used when training Model A and B, and the artery_shen_model, respectively.

The weight files for the Artery and Shen Model, Model A, and Model B, have been included as artery_shen_weights.pt, model_a.pt, and model_b.pt, respectively, for the exact weights used in the paper. These can be used in Function 7 to output the exact same figures shown in the paper.

### Table of Contents

- [**Function 1**](#function1)  
  Augment Image Contrast

- [**Function 2**](#function2)  
  Divide Images and Labels into Multiple Subsections
  
- [**Function 3**](#function3)  
  Combine Artery Dataset Classes
  
- [**Function 4**](#function4)  
  Resize Input Images
  
- [**Function 5**](#function5)  
  Split Data into Training and Validation Sets
  
- [**Function 6**](#function6)  
  Train Models A and B

- [**Function 7**](#function7)  
  Predict with Models
  
- [**Function 8**](#function8)  
  Generate Mask Images
  
- [**Function 9**](#function9)  
  Calculate Metrics from Binary Masks
  
- [**Function 10**](#function10)  
  Calculate Noise Level of Images
  

### Augment Image Contrast (Function 1) <a class="anchor" id="function1"></a>

**Purpose**  
Changes the contrast of the inputed images to augment the training data. Only the `200kV_500kx_p2nm_8cmCL_grain1_0111` image is augmented, as it is significantly more loop-heavy than the other `Shen et al.` image re-labelled.


**Parameters**  
`img_dir`: The original image directory.

`contrast_dir`: The directory to save the images with altered contrast.

`label_dir`: The original label directory.

`contrast_label_dir`: The directory to save the labels corresponding to the new images with altered contrast. These labels are identical to the originals, but are renamed to correspond to the new images.

In [ ]:
import os
import cv2
import shutil
import numpy as np

def augment_image_contrast(img_dir, contrast_dir, label_dir, contrast_label_dir):
    # Create output directory if it doesn't exist
    os.makedirs(contrast_dir, exist_ok=True)
    os.makedirs(contrast_label_dir, exist_ok=True)
    
    # Get a list of all image files in the directory
    img_files = [f for f in os.listdir(img_dir) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpg')]
    for img_file in img_files:
        # Set paths
        img_path = os.path.join(img_dir, img_file)
        filename, file_extension = os.path.splitext(img_file)
        contrast_path= os.path.join(contrast_dir, f'{filename}-darkened.{file_extension}')
        
        # Copy label file to new path to correspond with new image
        label_path = os.path.join(label_dir, f'{filename}.txt')
        contrast_label_path = os.path.join(contrast_label_dir, f'{filename}-darkened.txt')
        shutil.copyfile(label_path, contrast_label_path)
        
        # Adjust contrast via cv2, adapted from https://docs.opencv.org/4.x/d3/dc1/tutorial_basic_linear_transform.html
        img = cv2.imread(img_path)
        contrast_img = np.zeros(img.shape, img.dtype)
        
        for y in range(img.shape[0]):
            for x in range(img.shape[1]):
                for c in range(img.shape[2]):
                    contrast_img[y,x,c] = np.clip(2.0*img[y,x,c] - 200, 0, 255)
        
        cv2.imwrite(contrast_path, contrast_img)

In [ ]:
augment_image_contrast("./nature_img_used/original_images/shen/200kV_500kx_p2nm_8cmCL_grain1_0111/", "./nature_img_used/original_images/shen/200kV_500kx_p2nm_8cmCL_grain1_0111/", 
                       "./nature_img_used/original_images/shen/200kV_500kx_p2nm_8cmCL_grain1_0111/", "./nature_img_used/original_images/shen/200kV_500kx_p2nm_8cmCL_grain1_0111/")

### Divide Images and Labels into Multiple Subsections (Function 2) <a class="anchor" id="function2"></a>

**Purpose**  
Divides the images and corresponding labels downloaded from their sources into multiple smaller subsections to be used in training. This helps augment the training by generating more images to train on. The `200kV_500kx_p2nm_8cmCL_grain1_0111` image from `Shen et al.` is divided into the most training images due to being significantly more loop-heavy than the other Shen et al. image re-labelled, and the `MA956` images are divided into fewer as there are more images labelled in the YOLO format to train on to begin with. 


**Parameters**  
`img_dir`: The original image directory.  

`label_dir`: The label directory. The labels must be in YOLO format.  

`divided_img_dir`: The output directory to save the divided images.

`divided_label_dir`: The output directory to save the divided labels.

`dimension`: The number of vertical and horizontal segments to divide each image into (e.g. 3 would split the images into 3x3 segments)

In [ ]:
import os
import shutil
import cv2
import numpy as np
from random import randint
from shapely.geometry import Polygon
import matplotlib.pyplot as plt


def divide_images_and_labels(img_dir, label_dir, divided_img_dir, divided_label_dir, dimension):
    # Create output directories if they don't exist
    os.makedirs(divided_img_dir, exist_ok=True)
    os.makedirs(divided_label_dir, exist_ok=True)

    divide_ratio = dimension  # Ratio to divide image into x-by-x (e.g., 2x2 grid)

    # Get a list of all image files in the directory
    img_files = [f for f in os.listdir(img_dir) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpg')]

    # Iterate through each image file
    for img_file in img_files:
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, f'{os.path.splitext(img_file)[0]}.txt')
        
        # Add full image/label to the directory to train on too
        output_img_path = os.path.join(divided_img_dir, img_file)
        output_label_path = os.path.join(divided_label_dir, f'{os.path.splitext(img_file)[0]}.txt')
        shutil.copyfile(img_path, output_img_path)
        shutil.copyfile(label_path, output_label_path)

        # Read the image
        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        # Define divide size (e.g., half of the width and height for a 2x2 grid)
        divide_width, divide_height = w // divide_ratio, h // divide_ratio

        # Read labels from the corresponding label file
        with open(label_path, 'r') as f:
            labels = f.read().splitlines()

        # Process each divide
        for i in range(divide_ratio):  # For horizontal splits
            for j in range(divide_ratio):  # For vertical splits
                x_offset, y_offset = i * divide_width, j * divide_height
                divide = img[y_offset:y_offset + divide_height, x_offset:x_offset + divide_width]

                # Process labels for this divide
                divided_labels = []
                for label in labels:
                    class_id, *poly = label.split(' ')
                    poly = np.asarray(poly, dtype=np.float32).reshape(-1, 2)
                    poly *= [w, h]

                    # Clip a polygon to the boundaries of a divide area defined by x_offset, y_offset, divide_width, divide_height
                    # Create a polygon representing the divide area
                    divide_rect = Polygon([(x_offset, y_offset), (x_offset + divide_width, y_offset), 
                                         (x_offset + divide_width, y_offset + divide_height), (x_offset, y_offset + divide_height)])

                    # Create a polygon from the input polygon coordinates
                    label_poly = Polygon(poly.reshape(-1, 2))

                    # Fix the polygon if it's invalid
                    if not label_poly.is_valid:
                        label_poly = label_poly.buffer(0)

                    # Calculate the intersection of the label polygon with the divide area
                    clipped_poly = label_poly.intersection(divide_rect)

                    # If the intersection is not empty, return the coordinates of the clipped polygon
                    if not clipped_poly.is_empty:
                        if clipped_poly.geom_type == 'MultiPolygon':
                            # Handle MultiPolygon case by concatenating coordinates of all polygons
                            coords = np.concatenate([np.array(poly.exterior.coords.xy).T for poly in clipped_poly.geoms])
                        else:
                            # Single Polygon case, return its coordinates
                            coords = np.array(clipped_poly.exterior.coords.xy).T
                        clipped_poly = coords
                    else: 
                        clipped_poly = np.array([]).reshape(0, 2)

                    if clipped_poly.size > 0:
                        # Adjust polygon coordinates relative to a divide area and convert them to ratios
                        # Adjust the polygon coordinates to the divided image's coordinate system
                        adjusted_poly = clipped_poly.copy()
                        adjusted_poly[:, 0] -= x_offset
                        adjusted_poly[:, 1] -= y_offset

                        # Convert the coordinates to ratios relative to the divided image
                        adjusted_poly[:, 0] = np.clip(adjusted_poly[:, 0] / divide_width, 0, 1)
                        adjusted_poly[:, 1] = np.clip(adjusted_poly[:, 1] / divide_height, 0, 1)

                        # Save polygons as ratios in the label file
                        divided_labels.append(f"{class_id} {' '.join(map(str, adjusted_poly.flatten()))}")

                # Save the divided image
                divided_img_filename = f"{os.path.splitext(img_file)[0]}_{i}_{j}.png"
                divided_img_path = os.path.join(divided_img_dir, divided_img_filename)
                cv2.imwrite(divided_img_path, divide)

                # Save the divided label
                divided_label_filename = f"{os.path.splitext(img_file)[0]}_{i}_{j}.txt"
                divided_label_path = os.path.join(divided_label_dir, divided_label_filename)
                with open(divided_label_path, 'w') as f:
                    for divided_label in divided_labels:
                        f.write(f"{divided_label}\n")

    print("Images and labels are divided")

In [ ]:
# Divides the Shen et al. images
divide_images_and_labels("./nature_img_used/original_images/shen/1ROI_100kx_4100CL_foil1/", "./nature_img_used/original_images/shen/1ROI_100kx_4100CL_foil1", "./nature_img_used/augmented/shen_divided/images/", "./nature_img_used/augmented/shen_divided/labels/", 2)
divide_images_and_labels("./nature_img_used/original_images/shen/200kV_500kx_p2nm_8cmCL_grain1_0111/", "./nature_img_used/original_images/shen/200kV_500kx_p2nm_8cmCL_grain1_0111/", "./nature_img_used/augmented/shen_divided/images/", "./nature_img_used/augmented/shen_divided/labels/", 4)

# Divides the MA956 ODS dataset
divide_images_and_labels("./nature_img_used/original_images/ma956/", "./nature_img_used/original_images/ma956/", "./nature_img_used/augmented/ma956_divided/images/", "./nature_img_used/augmented/ma956_divided/labels/", 3)

### Combine Artery Dataset Classes (Function 3) <a class="anchor" id="function3"></a>

**Purpose**  
The `Artery` dataset labels define multiple classes of arteries. For the purpose of this study, all artery classes are changed to `0` (lines).

**Parameters**  
`label_dir`: The orginal label directory for the Artery dataset with differing Artery classes. 

`output_dir`: The output Artery label directory with a single Artery/line class.

In [ ]:
import os


def combine_classes(label_dir, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(label_dir):
        if filename.endswith('.txt'):
            input_path = os.path.join(label_dir, filename)
            output_path = os.path.join(output_dir, filename)

            with open(input_path, 'r') as f:
                lines = f.readlines()

            with open(output_path, 'w') as bbox_file:
                for line in lines:
                    numbers = line.split()
                    numbers[0] = '0'  # Change the first number to 0
                    new_line = ' '.join(numbers)  # Join the numbers back into a string
                    bbox_file.write(new_line + '\n')  # Write the modified line to the output file

In [ ]:
combine_classes("./nature_img_used/original_images/artery/", "./nature_img_used/augmented/artery_combined/labels/")

### Resize Input Images (Function 4) <a class="anchor" id="function4"></a>

**Purpose**  
The `Artery` dataset images are 512x512. Function 1 will divide the `Shen et al.` images to a different size. This resizes them to 512x512 to provide a uniform image size for training. The `MA956` images are not resized, as the model performed better training on the original-sized images output from Function 1.

**Parameters**  
`divided_img_dir`: Directory with the divided Shen et al. images.

`resized_img_dir`: Directory where resized images will be saved.

In [ ]:
import os
import cv2


def resize_images(divided_img_dir, resized_img_dir):
    # Create output directory if it doesn't exist
    os.makedirs(resized_img_dir, exist_ok=True)

    img_files = [f for f in os.listdir(divided_img_dir) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpg')]
    for img_file in img_files:
        # Construct paths for the current image file
        img_path = os.path.join(divided_img_dir, img_file)
        output_img_path = os.path.join(resized_img_dir, img_file)
  
        # Read image
        image = cv2.imread(img_path)
        # Resize image to 512x512 if size is not already 512x512
        if image.shape[:2] != (512, 512):
            image = cv2.resize(image, (512, 512))

        # Write resized image
        cv2.imwrite(output_img_path, image)

    print("Images are resized")

In [ ]:
resize_images("./nature_img_used/augmented/shen_divided/images/", "./nature_img_used/augmented/shen_resized/images/")

### Split Data into Training and Validation Sets (Function 5) <a class="anchor" id="function5"></a>

**Purpose**  
Randomly splits dataset images and corresponding labels into training and validation directories for model training. This is also used to match the number of `Artery` images to those of the post-processed `Shen et al.` ones. From testing, a train-val split of 80-20 was optimal for the Artery and Shen Model, while 90-10 performed better for the `MA956` model.


**Parameters**  
`img_dir`: The image directory of the full dataset.  

`label_dir`: The label directory of the full dataset.  

`train_img_dir`: The output directory to save the training images.

`val_img_dir`: The output directory to save the validation images.

`train_label_dir`: The output directory to save the training labels.

`val_label_dir`: The output directory to save the validation labels.

`val_ratio`: The ratio of images/labels to be used for validation to the total number in the dataset.

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split


def split_train_val(img_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, val_ratio):
    # Create output directories if they don't exist
    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)
    
    img_files = [os.path.join(img_dir, filename) for filename in os.listdir(img_dir) if filename.endswith('.tif') or filename.endswith('.png') or filename.endswith('.jpg')]
    label_files = [os.path.join(label_dir, filename) for filename in os.listdir(label_dir) if filename.endswith('.txt')]

    # Sort the filenames to ensure correspondence between images and labels
    img_files.sort()
    label_files.sort()

    # Perform train-test split using sklearn.model_selection.
    train_imgs, val_imgs, train_labels, val_labels = train_test_split(img_files, label_files, test_size=val_ratio, random_state=42)

    # Copy train images to the training directory
    for img_path in train_imgs:
        img_filename = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(train_img_dir, img_filename))

    # Copy train labels to the training directory
    for label_path in train_labels:
        label_filename = os.path.basename(label_path)
        shutil.copy(label_path, os.path.join(train_label_dir, label_filename))

    # Copy val images to the validation directory
    for img_path in val_imgs:
        img_filename = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(val_img_dir, img_filename))

    # Copy val labels to the validation directory
    for label_path in val_labels:
        label_filename = os.path.basename(label_path)
        shutil.copy(label_path, os.path.join(val_label_dir, label_filename))

    # Check the lengths of the training and testing sets
    print("Training set size:", len(train_imgs))
    print("Validation set size:", len(val_imgs))

In [ ]:
# We only need 39 artery images to match the 39 from Shen et al. post-divide
split_train_val("./nature_img_used/original_images/artery/", "./nature_img_used/augmented/artery_combined/labels/", 
                "./nature_img_used/augmented/artery_extra/", "./nature_img_used/augmented/artery_used/images/",
               "./nature_img_used/augmented/artery_extra/", "./nature_img_used/augmented/artery_used/labels/", 0.13)


# Set up training and validation directories for the artery/Shen model
split_train_val("./nature_img_used/augmented/shen_resized/images/", "./nature_img_used/augmented/shen_divided/labels/", 
                "./nature_img_used/post-processed_images/artery_shen_model/images/train/", "./nature_img_used/post-processed_images/artery_shen_model/images/val/",
               "./nature_img_used/post-processed_images/artery_shen_model/labels/train/", "./nature_img_used/post-processed_images/artery_shen_model/labels/val/", 0.2)

split_train_val("./nature_img_used/augmented/artery_used/images/", "./nature_img_used/augmented/artery_used/labels/", 
                "./nature_img_used/post-processed_images/artery_shen_model/images/train/", "./nature_img_used/post-processed_images/artery_shen_model/images/val/",
               "./nature_img_used/post-processed_images/artery_shen_model/labels/train/", "./nature_img_used/post-processed_images/artery_shen_model/labels/val/", 0.2)

# Set up training and validation directories for models a and b
split_train_val("./nature_img_used/augmented/ma956_divided/images/", "./nature_img_used/augmented/ma956_divided/labels/", 
                "./nature_img_used/post-processed_images/models_a_b/images/train/", "./nature_img_used/post-processed_images/models_a_b/images/val/",
               "./nature_img_used/post-processed_images/models_a_b/labels/train/", "./nature_img_used/post-processed_images/models_a_b/labels/val/", 0.1)

### Train Models A and B (Function 6) <a class="anchor" id="function6"></a>

**Purpose**  
Trains the models on the provided images with the same parameters used in the paper.   

**Instructions**

This function makes use of the `nature_models.yaml` file found in `nature_img_used/nature_models.yaml`. Users will need to specify their working directory (i.e. where they unzipped the `nature_img_used.zip` file) for the `path` variable. The two different lines with the `path` variable correspond to the two different training datasets used in training models A and B, and the Artery and Shen Model used to pretrain `Model B`. The correct line will need to be uncommented and the other remain commented when training those respective models.

**Parameters**  

`base_model`: The training model. YOLO can train from scratch, pre-trained weights from MSCOCO, or weight files specified from previous runs.

`imgsz`: The size of the images in the training data. 

`epochs`: The number of training passes over the entire training dataset. 

`batch`: The number of training images to be processed before updating model weights. 

`lr0`: The initial learning rate of the model, which affects how much model weights can be updated after each image batch. 

`patience`: The number of epochs with no model improvements to wait before terminating the training. 

`project`: The output path for model runs.  



**YOLOv8 Training Parameters**  

`freeze`: The number of model layers whose weights are frozen for transfer learning.

`verbose`: Enabling provides detailed logs and progress updates during training.  

`plots`: Enabling outputs plots of training and validation metrics and prediction examples.   

`amp` and `half`: Enabling can reduce memory usage and speed up training with minimal impact on accuracy.  

In [ ]:
import os
import yaml
from ultralytics import YOLO
import tensorflow as tf

# Define the path to the YAML file
yaml_path = "./nature_img_used/nature_models.yaml"

def train_model(base_model, imgsz, epochs, batch, lr0, patience, project):
    # Load the model
    model = YOLO(base_model)
    
    # Generate Tensorboard graphs for monitoring during training
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
    
    model.train(
        data=yaml_path, 
        imgsz=imgsz,
        epochs=epochs,
        batch=batch,
        lr0=lr0,
        patience=patience,
        project=project,
        freeze=1,
        verbose=True,
        amp=True,
        plots=True,
        half=True
    )

In [ ]:
train_model('yolov8x-seg.pt', 640, 100, 32, 0.01, 100, "./model-a")

In [ ]:
train_model('yolov8x-seg.pt', 512, 200, 16, 0.001, 50, "./artery_shen")

In [ ]:
train_model('./artery_shen/train/weights/best.pt', 640, 100, 32, 0.01, 50, "./model-b")

### Predict with Models (Function 7)  <a class="anchor" id="function7"></a>


**Purpose**  
Predicts on the test images using the trained models and outputs mask label files.  

**Parameters**  
`model_weights_path`: The path to the best weights of the trained models. 

`test_img_dir`: The test image directory.   

`project`: The output path for model predictions.

`conf`: The minimum confidence threshold of predictions to output.

**YOLOv8 Prediction Parameters**  

`max_det`: The maximum dislocation instances detected.

`iou`: The maximum Intersection Over Union (IoU) threshold between different predictions.

`show_labels`: Disabling removes text labels for every prediction instance in output images.

`show_boxes`: Disabling removes bounding boxes for every prediction instance in output images, and leaves only the segmentations. 

`save`: Enabling saves the prediction images.

`save_txt`: Enabling saves the predictions in a .txt file of YOLO format.

In [ ]:
from ultralytics import YOLO
import glob

def predict(model_weights_path, test_img_dir, project, conf):
    model = YOLO(model_weights_path)
    
    #Specify image directory model predicts on
    test_img_dir = glob.glob(test_img_dir) 

    for img in test_img_dir:
        model.predict(
            source=img, 
            project=project, 
            conf=conf, 
            max_det=2500, 
            iou=0.7,
            show_labels=False, 
            show_boxes=False, 
            save=True, 
            save_txt=True
        )

In [ ]:
predict("./model-a/train/weights/best.pt", "./nature_img_used/test_img/images/", "./model-a/", 0.04)
predict("./model-b/train/weights/best.pt", "./nature_img_used/test_img/images/", "./model-b/", 0.04)

# Predict on Shen et al.'s images
predict("./model-a/train/weights/best.pt", "./nature_img_used/shen_test/images/", "./model-a/", 0.04)
predict("./model-b/train/weights/best.pt", "./nature_img_used/shen_test/images/", "./model-b/", 0.04) 

# Predict on HfAl with lower confidence threshold
predict("./model-b/train/weights/best.pt", "./nature_img_used/hfal/", "./model-b/", 0.01) 

### Generate Mask Images (Function 8) <a class="anchor" id="function8"></a>

**Purpose**  
Plots the masks from the label text files. Can do YOLO format as well as `Shen et al.`'s bounding boxes. Can overlay the masks on top of the original image, or create binary mask images of just the masks on a black background for metric calculation.  


**Parameters**  
`img_dir`: The unlabeled image directory.  

`label_dir`: The label directory. The labels can be either the ground truth or model predictions, and must be in YOLO or Shen et al.'s format depending on the `label_format` specified.  

`output_dir`: The output directory to save the mask images.

`mode`: `overlay` (default) or `metric` depending on whether you want to overlay the masks on the original images, or plot just the binary masks for metric calculation, respectively.

`label_format`: `yolo` (default) or `shen` depending on whether the label files provided are in YOLO format or `Shen et al.`'s format.

In [ ]:
import numpy as np
import os
import cv2

def plot_masks(img_dir, label_dir, output_dir, mode='overlay', label_format='yolo'):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Define colors for masks
    line_color = (0, 255, 0)  # Green for lines
    loop_color = (0, 0, 255)   # Red for loops
    bb_color = (255, 255, 0)  # Cyan for Shen et al.'s bounding boxes

    # Get a list of all image files in the directory
    img_files = [f for f in os.listdir(img_dir) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpg')]

    # Process each image file
    for img_file in img_files:
        # Construct paths for the current image and label file
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, f'{os.path.splitext(img_file)[0]}.txt')
        output_img_path = os.path.join(output_dir, img_file)

        # Check if the image and label files exist
        if not os.path.exists(img_path):
            print(f"Image file '{img_path}' not found. Skipping...")
            continue

        if not os.path.exists(label_path):
            print(f"Label file '{label_path}' not found. Skipping...")
            continue

        # Read the image
        img = cv2.imread(img_path)
        h, w = img.shape[:2]  # Get height and width of the image

        if mode != 'metric' and label_format != 'shen':  # Essentially just for YOLO format overlays but make that the default
            # Create two blank mask images with fully transparent backgrounds - one for each label to combine later
            line_mask = np.zeros((h, w, 4), dtype=np.uint8)  # 4 channels for RGBA (Red, Green, Blue, Alpha)
            loop_mask = np.zeros((h, w, 4), dtype=np.uint8)
        else:
            # Create two blank black images - one for each label to combine later
            line_mask = np.zeros((h, w, 3), dtype=np.uint8)  # 3 channels for RGBA (Red, Green, Blue)
            loop_mask = np.zeros((h, w, 3), dtype=np.uint8)

        # Read labels from the corresponding label file
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Loop though all masks and plot them
        for line in lines:
            data = line.strip().split(' ')
            points = list(map(float, data[1:]))
            
            if label_format == 'shen':
                y_min, x_min, y_max, x_max = map(int, points)  # This is the format Shen et al.'s labels used
                if mode == 'metric':
                    cv2.rectangle(loop_mask, (x_min, y_min), (x_max, y_max), color=loop_color, thickness=-1)
                else:  # overlay
                    cv2.rectangle(loop_mask, (x_min, y_min), (x_max, y_max), color=bb_color, thickness=4)
            
            else:  # YOLO format
                class_id = int(data[0])
                num_points = len(points) // 2
                if num_points < 3:  # Need at least 3 points to define a polygon
                    continue
                polygon = np.array(points, dtype=np.float32).reshape(-1, 2)
                polygon[:, 0] *= w
                polygon[:, 1] *= h
                polygon = polygon.astype(np.int32)

                if mode == 'metric':
                    if class_id == 0:
                        cv2.fillPoly(line_mask, [polygon], color=line_color)
                    elif class_id == 1:
                        cv2.fillPoly(loop_mask, [polygon], color=loop_color)
                else:  # overlay
                    if class_id == 0:
                        cv2.fillPoly(line_mask, [polygon], color=line_color + (100,))  # Adjust alpha value (100 for semi-transparency)
                    elif class_id == 1:
                        cv2.fillPoly(loop_mask, [polygon], color=loop_color + (100,))
                
        
        # Combine the mask images into one, so that any overlap shows up as both classes
        mask = cv2.add(line_mask, loop_mask)

        if mode == 'metric':
            output = mask
        else:
            if label_format == 'shen':
                # Plot bounding boxes on Shen et al. images, since they are supposed to be solid this is a hacky way to not blend the other pixel colors
                loop_grayscale = cv2.cvtColor(loop_mask,cv2.COLOR_BGR2GRAY)
                mask = cv2.threshold(loop_grayscale, 10, 255, cv2.THRESH_BINARY)[1]
                img_background = cv2.bitwise_and(img, img, mask = cv2.bitwise_not(mask))
                output = cv2.add(img_background, loop_mask)
            else:
                # Overlay the mask on the original image using the mask as an alpha channel
                img[:, :, :3] = cv2.addWeighted(img, 1, mask[:, :, :3], 0.3, 0)
                output = img

        # Save the overlay image with the masks and polylines
        cv2.imwrite(output_img_path, output)

In [ ]:
# Overlaid masks to be used as paper figures
# Ground truths
plot_masks(img_dir="./nature_img_used/test_img/images/", label_dir="./nature_img_used/test_img/labels/", output_dir="./paper_masks/gt/", mode='overlay', label_format='yolo')

# Models A and B
plot_masks(img_dir="./nature_img_used/test_img/images/", label_dir="./model-a/predict/labels/", output_dir="./paper_masks/a/", mode='overlay', label_format='yolo')
plot_masks(img_dir="./nature_img_used/test_img/images/", label_dir="./model-b/predict/labels/", output_dir="./paper_masks/b/", mode='overlay', label_format='yolo')

# Models A and B on Shen et al.'s images
plot_masks(img_dir="./nature_img_used/shen_test/images/", label_dir="./model-a/predict2/labels/", output_dir="./paper_masks/a/", mode='overlay', label_format='yolo')
plot_masks(img_dir="./nature_img_used/shen_test/images/", label_dir="./model-b/predict2/labels/", output_dir="./paper_masks/b/", mode='overlay', label_format='yolo')

# Masks for HfAl image
plot_masks(img_dir="./nature_img_used/hfal/", label_dir="./model-b/predict3/labels/", output_dir="./paper_masks/b/", mode='overlay', label_format='yolo')

In [ ]:
# Shen et al.'s bounding boxes
plot_masks(img_dir="./nature_img_used/shen_test/images/", label_dir="./nature_img_used/shen_test/labels/", output_dir="./paper_masks/shen/", mode='overlay', label_format='shen')

In [ ]:
# Binary mask images for metric calculation
# Ground truths
plot_masks(img_dir="./nature_img_used/test_img/images/", label_dir="./nature_img_used/test_img/labels/", output_dir="./binary_masks/gt/", mode='metric', label_format='yolo')

# Models A and B
plot_masks(img_dir="./nature_img_used/test_img/images/", label_dir="./model-a/predict/labels/", output_dir="./binary_masks/a/", mode='metric', label_format='yolo')
plot_masks(img_dir="./nature_img_used/test_img/images/", label_dir="./model-b/predict/labels/", output_dir="./binary_masks/b/", mode='metric', label_format='yolo')

# Models A and B on Shen et al.'s images
plot_masks(img_dir="./nature_img_used/shen_test/images/", label_dir="./model-a/predict2/labels/", output_dir="./binary_masks/a/", mode='metric', label_format='yolo')
plot_masks(img_dir="./nature_img_used/shen_test/images/", label_dir="./model-b/predict2/labels/", output_dir="./binary_masks/b/", mode='metric', label_format='yolo')

# Shen et al.'s bounding boxes
plot_masks(img_dir="./nature_img_used/shen_test/images/", label_dir="./nature_img_used/shen_test/labels/", output_dir="./binary_masks/shen/", mode='metric', label_format='shen')

###  Calculate Metrics from Binary Masks (Function 9) <a class="anchor" id="function9"></a>
**Purpose**  
Calculates the Precision, Recall, F1 Score, and IOU metrics from binary mask images using boolean logic.

**Parameters**  

`gt_dir`: The directory containing ground truth binary mask images. 

`prediction_dir`: The directory containing model-predicted binary mask images. 

`model_name`: Name of the model predicted with.  

`dislocation_class`: `loops` or `lines`, corresponding to which class metrics should be outputted for.  

In [ ]:
import os
import numpy as np
from PIL import Image

def calculate_metrics(gt_dir, prediction_dir, model_name, dislocation_class):
    # Sets the dimension of rbg pixels to look at depending on whether we want metrics for loops or lines
    if dislocation_class == 'lines':
        rbg_dim = 1
    else:
        rbg_dim = 0
    
    img_files = [f for f in os.listdir(gt_dir) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpg')]
    for img_name in img_files:
        GT_path = os.path.join(gt_dir, img_name)
        pred_path = os.path.join(prediction_dir, img_name)
        
        # Read in images
        img_GT = Image.open(GT_path)
        img_pred = Image.open(pred_path)
        
        # Convert pixel values to binary based on whether they contain any non-zero value
        gt_bbox = np.array(img_GT) > 0
        pred_mask = np.array(img_pred) > 0

        gt_bbox = gt_bbox[:,:,rbg_dim]
        pred_mask = pred_mask[:,:,rbg_dim]
        
        # Calculate true positives (tp), false positives (fp), and false negatives (fn) using boolean logical operators
        tp = np.sum(np.logical_and(gt_bbox, pred_mask))
        fp = np.sum(np.logical_and(np.logical_not(gt_bbox), pred_mask))
        fn = np.sum(np.logical_and(gt_bbox, np.logical_not(pred_mask)))
        
        # Calculate IOU, precision, recall, and f1 score, add 1e-9 so we don't have a divide by zero
        precision = tp / (tp + fp + 1e-9)
        recall = tp / (tp + fn + 1e-9)
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
        iou = tp / (tp + fp + fn + 1e-9)

        # Print the results
        print("\nThe precision of predicted " + dislocation_class + " from " + model_name + " for sample " + img_name + " is: " + str(precision))
        print("The recall of predicted " + dislocation_class + " from " + model_name + " for sample " + img_name + " is: " + str(recall))
        print("The F1 Score of predicted " + dislocation_class + " from " + model_name + " for sample " + img_name + " is: " + str(f1_score))
        print("The IOU of predicted " + dislocation_class + " from " + model_name + " for sample " + img_name + " is: " + str(iou))

In [ ]:
calculate_metrics("./binary_masks/gt/", "./binary_masks/a/", "Model A", "loops")
calculate_metrics("./binary_masks/gt/", "./binary_masks/a/", "Model A", "lines")
calculate_metrics("./binary_masks/gt/", "./binary_masks/b/", "Model B", "loops")
calculate_metrics("./binary_masks/gt/", "./binary_masks/b/", "Model B", "lines")
calculate_metrics("./binary_masks/shen/", "./binary_masks/a/", "Model A", "loops")
calculate_metrics("./binary_masks/shen/", "./binary_masks/b/", "Model B", "loops")

### Calculate Noise Level of Images (Function 10) <a class="anchor" id="function10"></a>
**Purpose**  
Quantifies the noisiness of images, where higher values correspond to higher noise in an image.

**Parameters**  

`img_dir`: Directory of the images to calculate the noise level of.  

In [ ]:
import os
import cv2
import numpy as np

def noise_level(img_dir):
    img_files = [f for f in os.listdir(img_dir) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpg')]
    for img_name in img_files:
        img_path = os.path.join(img_dir, img_name)
        
        # Load the image
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Calculate standard deviation of pixel values
        std_dev = np.std(image)

        print(f"The standard deviation of pixel values of image {img_name} is {std_dev}\n")

In [ ]:
# Calculate noise levels of Shen et al. test images
noise_level('./nature_img_used/shen_test/images/')

# Calculate noise levels of MA956 test images
noise_level('./nature_img_used/test_img/images/')

# Calculate noise levels of HfAl image
noise_level('./nature_img_used/hfal/')